### Version 1

In [ ]:
!pip install opencv-python pyzbar numpy pdf2image

In [ ]:
!apt-get update
!apt-get install libzbar0

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,315 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,674 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

In [ ]:
import cv2
import numpy as np
from pyzbar.pyzbar import decode

def scan_qr_code(image_path):
    # Load image
    image = cv2.imread(image_path)

    if image is None:
        return "Error: Image not found or could not be loaded."

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect and decode QR codes
    qr_codes = decode(gray)

    results = []
    for qr in qr_codes:
        qr_data = qr.data.decode("utf-8")
        results.append(qr_data)

        # Draw rectangle around QR code
        points = np.array(qr.polygon, np.int32)
        points = points.reshape((-1, 1, 2))
        cv2.polylines(image, [points], True, (0, 255, 0), 2)

        # Put decoded text near QR code
        x, y, w, h = qr.rect
        cv2.putText(image, qr_data, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the result image (optional)
    cv2.imwrite("qr_detected.png", image)

    return results if results else "No QR code found."

def get_product_details(product_id):
    url = f"http://qr.aristino.com/check.aspx?ID={product_id}"

    # Send a request to the URL
    response = requests.get(url)

    if response.status_code != 200:
        return {"error": "Failed to access the website"}

    # Parse the HTML content
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract product name
    product_name = soup.find("span", {"id": "Label_TenSP"})

    # Extract product code from input field
    product_code_input = soup.find("input", {"id": "LbInfo"})
    product_code = product_code_input["value"] if product_code_input else "Not found"

    return {
        "name": product_name.text.strip() if product_name else "Not found",
        "code": product_code
    }

# Example usage
image_path = "/content/test2.png"
result = scan_qr_code(image_path)
print(result)
qr_results = scan_qr_code(image_path)

if qr_results:
    for qr_data in qr_results:
        print(f"Fetching details for: {qr_data}")
        product_details = get_product_details(qr_data)
        print(product_details)
else:
    print("No QR code found.")

['http://qr.aristino.com/check.aspx?ID=SNT00400KPE00G033000']
Fetching details for: http://qr.aristino.com/check.aspx?ID=SNT00400KPE00G033000
{'name': '', 'code': 'http://qr.aristino.com/check.aspx?ID=SNT00400KPE00G033000'}


In [ ]:
import cv2
import numpy as np
import requests
from pyzbar.pyzbar import decode
from bs4 import BeautifulSoup

def scan_qr_code(image_path):
    # Load image
    image = cv2.imread(image_path)

    if image is None:
        return "Error: Image not found or could not be loaded."

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect and decode QR codes
    qr_codes = decode(gray)

    results = []
    for qr in qr_codes:
        qr_data = qr.data.decode("utf-8")

        # Extract product ID from the full URL if necessary
        if "ID=" in qr_data:
            product_id = qr_data.split("ID=")[-1]  # Extract only the ID
        else:
            product_id = qr_data

        results.append(product_id)

    return results if results else None

def get_product_details(product_id):
    url = f"http://qr.aristino.com/check.aspx?ID={product_id}"

    # Send a request to the URL
    response = requests.get(url)

    if response.status_code != 200:
        return {"error": "Failed to access the website"}

    # Parse the HTML content
    soup = BeautifulSoup(response.text, "html.parser")

    # Debugging: Print extracted HTML
    print("Debug: HTML Content Snippet:", soup.prettify()[:1000])  # Print first 1000 characters of the HTML

    # Extract product name
    product_name_tag = soup.find("span", {"id": "Label_TenSP"})
    product_name = product_name_tag.text.strip() if product_name_tag else "Not found"

    # Extract product code from input field
    product_code_input = soup.find("input", {"id": "LbInfo"})
    product_code = product_code_input["value"] if product_code_input else "Not found"

    return {
        "name": product_name,
        "code": product_code
    }

# Example usage
image_path = "/content/test1.58.30.png"  # Update this with your actual image path
qr_results = scan_qr_code(image_path)

if qr_results:
    for product_id in qr_results:
        print(f"Fetching details for Product ID: {product_id}")
        product_details = get_product_details(product_id)
        print("Product Details:", product_details)
else:
    print("No QR code found.")


Fetching details for Product ID: SNT00400KPE00B099000
Debug: HTML Content Snippet: <!DOCTYPE html>
<html>
 <head>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="Content/css/w3.css" rel="stylesheet"/>
  <style type="text/css">
   .auto-style1 {
            font-size: x-large;
        }
        .auto-style2 {
            width: 100%;
        }
        .auto-style3 {
            width: 70px;
        }
  </style>
 </head>
 <body>
  <form action="./check.aspx?ID=SNT00400KPE00B099000" id="form1" method="post" name="form1">
   <input id="ScriptManager1_TSM" name="ScriptManager1_TSM" type="hidden" value=""/>
   <input id="__VIEWSTATE" name="__VIEWSTATE" type="hidden" value="pEjMsHxs4nPionMTbTuBpF5AFt32BYuFc+IMV63PXkk7TSoNTQppGPKX+QM1Q9GPzjttuAZGC3rxKzRj//e3hFjXIEPiIMgJnnIXCGjexmpNfvG3FpIJEUd/xhU9UCk14L/f5PIltvK9M5DxJ/U3fePXyUZfrUFw73c4ZCHWOvzjAyLKhTEa971cOgIW2CnRs2aa+VUEAu/5oZG5ZClpq2tXgds11ExI4+6b5UBd6nUIZFvdDiMFQnFVDn+8CI2tBo1VvUVoGNbGx9zs8Jb1Foqb37LZJ5

In [ ]:
import cv2
import numpy as np
import requests
from pyzbar.pyzbar import decode
from bs4 import BeautifulSoup

def scan_qr_code(image_path):
    """Scans a QR code from an image and extracts the product ID."""
    image = cv2.imread(image_path)

    if image is None:
        print("Error: Image not found or could not be loaded.")
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    qr_codes = decode(gray)

    if not qr_codes:
        print("No QR code found.")
        return None

    product_ids = []
    for qr in qr_codes:
        qr_data = qr.data.decode("utf-8")

        if "ID=" in qr_data:
            product_id = qr_data.split("ID=")[-1]
        else:
            product_id = qr_data

        product_ids.append(product_id)

    return product_ids

def get_product_details(product_id):
    """Fetches product details from the website using the product ID."""
    url = f"http://qr.aristino.com/check.aspx?ID={product_id}"

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raise an error for bad responses (4xx, 5xx)
    except requests.RequestException as e:
        print(f"Error fetching product details: {e}")
        return {"error": "Failed to access the website"}

    soup = BeautifulSoup(response.text, "html.parser")
    product_name_tag = soup.find("span", {"id": "Label_TenSP"})
    product_name = product_name_tag.text.strip() if product_name_tag else "Not found"
    product_code_input = soup.find("input", {"id": "LbInfo"})
    product_code = product_code_input["value"].strip() if product_code_input else "Not found"

    return {"name": product_name, "code": product_code}

image_path = "/content/test2.png"
qr_results = scan_qr_code(image_path)

if qr_results:
    for product_id in qr_results:
        print(f"\nFetching details for Product ID: {product_id}")
        product_details = get_product_details(product_id)
        print("Product Details:", product_details)
else:
    print("No valid QR codes detected.")



Fetching details for Product ID: SNT00400KPE00G033000
Product Details: {'name': 'Khăn Salina SNT004 màu Xám 33', 'code': 'SNT00400KPE00G033000'}


### Version 2


In [10]:
!apt-get install poppler-utils
!pip install opencv-python pyzbar numpy pdf2image
!apt-get install libzbar0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libzbar0 is already the newest version (0.23.92-4build2).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [11]:
import cv2
import numpy as np
import requests
from pyzbar.pyzbar import decode
from pdf2image import convert_from_path
from bs4 import BeautifulSoup

def extract_images_from_pdf(pdf_path):
    images = convert_from_path(pdf_path, dpi=300)
    return images

def scan_qr_codes_from_image(image):
    open_cv_image = np.array(image)
    open_cv_image = cv2.cvtColor(open_cv_image, cv2.COLOR_RGB2BGR)

    gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
    qr_codes = decode(gray)

    product_ids = []
    for qr in qr_codes:
        qr_data = qr.data.decode("utf-8")

        if "ID=" in qr_data:
            product_id = qr_data.split("ID=")[-1]
        else:
            product_id = qr_data

        product_ids.append(product_id)

    return product_ids

def get_product_details(product_id):
    url = f"http://qr.aristino.com/check.aspx?ID={product_id}"

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching product details: {e}")
        return {"error": "Failed to access the website"}
    soup = BeautifulSoup(response.text, "html.parser")
    product_name_tag = soup.find("span", {"id": "Label_TenSP"})
    product_name = product_name_tag.text.strip() if product_name_tag else "Not found"
    product_code_input = soup.find("input", {"id": "LbInfo"})
    product_code = product_code_input["value"].strip() if product_code_input else "Not found"

    return {"name": product_name, "code": product_code}








pdf_path = "/content/SNT004MT_khandanang_sieumem_7mau.pdf"  # Update this with your actual PDF file path
images = extract_images_from_pdf(pdf_path)

all_product_ids = set()
for page_number, image in enumerate(images):
    print(f"Scanning QR codes from page {page_number + 1}...")
    product_ids = scan_qr_codes_from_image(image)
    all_product_ids.update(product_ids)

if all_product_ids:
    for product_id in all_product_ids:
        print(f"\nFetching details for Product ID: {product_id}")
        product_details = get_product_details(product_id)
        print("Product Details:", product_details)
else:
    print("No QR codes detected in the PDF.")


Scanning QR codes from page 1...

Fetching details for Product ID: SNT00400KPE00B099000
Product Details: {'name': 'Khăn Salina SNT004 màu Xanh biển 99', 'code': 'SNT00400KPE00B099000'}

Fetching details for Product ID: SNT00400KPE00Y030000
Product Details: {'name': 'Khăn Salina SNT004 màu Vàng 30', 'code': 'SNT00400KPE00Y030000'}

Fetching details for Product ID: SNT00400KPE00G033000
Product Details: {'name': 'Khăn Salina SNT004 màu Xám 33', 'code': 'SNT00400KPE00G033000'}

Fetching details for Product ID: SNT00400KPE00V02100
Product Details: {'name': 'Khăn Salina SNT004 màu Tím 21', 'code': 'SNT00400KPE00V02100'}

Fetching details for Product ID: SNT00400KPE00B034000
Product Details: {'name': 'Khăn Salina SNT004 màu Xanh biển 34', 'code': 'SNT00400KPE00B034000'}

Fetching details for Product ID: SNT00400KPE00E001000
Product Details: {'name': 'Khăn Salina SNT004 màu Be 01', 'code': 'SNT00400KPE00E001000'}

Fetching details for Product ID: SNT00400KPE00C099000
Product Details: {'name': 